## Step 20: 연산자 오버로드(1)

이번에 +와 * 연산자를 지원하도록 Variable을 확장할 것이다. 이번 단계에서는 곱셈을 수행하는 Mul 함수를 구현해보자.

### 20.1 Mul 클래스 구현

곱셈의 미분은 $y=x_0 * x_1$일 때 ${\delta y}\over {\delta x_0} = x_1$, ${\delta y}\over {\delta x_1} = x_0$가 된다.

따라서 역전파는 최종 출력(loss) $L$의 미분을 전파한다. 이때 변수 $x_0$와 $x_1$에 대한 미분은 각각 ${{\delta L}\over {\delta x_0}}=x_1{{\delta L}\over {\delta y}}$와 ${{\delta L}\over {\delta x_1}}=x_0{{\delta L}\over {\delta y}}$이다.

```python
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0
```

만들어진 Mul 클래스를 파이썬 함수로 사용할 수 있도록 하겠다.

```python
def mul(x0, x1):
    return Mul()(x0, x1)
```

이제 mul 함수를 이용하여 곱셈을 다음과 같이 수행할 수 있다.

```python
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a,b),c)

y.backward()

print(y)
print(a.grad)
print(b.grad)
```

\> variable(7.0)

\> 2.0

\> 3.0

이제 앞선 단계에서 구현했던 add나 square처럼, mul도 파이썬 함수처럼 사용할 수 있게 되었다. 하지만 매번 `add(mul(a,b),c)`처럼 작성하는 것은 번거롭다. 이런 표현 대신 a*b+c 꼴로 사용할 수 있도록 연산자 오버로드를 해보도록 하자. 연산자를 오버로드하면, 오버로드된 연산자 사용 시 사용자가 설정한 함수가 호출된다. 파이썬에서는 \_\_add__나 \_\_mul__ 같은 특수 메서드를 정의함으로써 사용자 지정 함수가 호출되도록 한다.

### 20.2 연산자 오버로드

먼저 곱셈 연산자 *를 오버로드해보자. 곱셈의 특수 메서드는 \_\_mul__(self, other)이다. 이 메서드를 정의하면 * 연산자를 사용할 때 해당 메서드가 호출되도록 오버로드 할 수 있다.

```python
class Variable:
    ...
    def __mul__(self, other):
        return mul(self, other)
```

```python
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
y = a * b
print(y)
```

\> variable(6.0)

위처럼 \_\_mul__ 메서드를 정의하면 y = a * b와 같은 코드를 문제 없이 사용할 수 있다. a * b가 실행될 때 인스턴스 a의 \_\_mul__ 메서드가 호출되고 a의 값은 self 인수로 전달된다. 동시에 b의 값이 other로 전달되어 계산이 수행된다.

위에서는 Variable 클래스 안에 \_\_mul__ 메서드를 구현하였다. 하지만 보다 간단하게 처리하는 방법도 있다.

```python
class Variable:
    ...

Variable.__mul__ = mul
Variable.__add__ = add
```

위처럼 Variable 클래스가 정의되어 있는 상태에서 Variable.\_\_mul__ = mul 과 같이 작성하면 된다. 파이썬에서는 함수도 객체이므로 이처럼 함수 자체를 할당할 수 있다.